# Performance optimization overview

This tutorial illustrates the performance optimizations applied to the code generated by an `Operator`. As we shall see, most optimizations are automatically applied as they're known to systematically improve performance. Others, whose impact varies across different `Operator`'s, are instead to be enabled through specific options.

An Operator has several preset **optimization levels**; the fundamental ones are `noop` and `advanced`. With `noop`, no performance optimizations are introduced by the compiler. With `advanced`, several flop-reducing and data locality optimization passes are performed. Examples of flop-reducing optimization passes are common sub-expressions elimination and factorization; examples of data locality optimization passes are loop fusion and cache blocking. SIMD vectorization, via compiler auto-vectorization, is enforced through OpenMP pragmas.

An optimization pass may provide knobs, or **options**, for fine-grained tuning. As explained in the next sections, some of these options are given at compile-time, others at run-time.

**\*\* Remark \*\***

Parallelism -- both shared-memory (e.g., OpenMP) and distributed-memory (MPI) -- is _by default disabled_ and is _not_ controlled via the optimization level. In this tutorial we will also show how to enable OpenMP parallelism (you'll see it's trivial!). A mini-guide about parallelism in Devito and related aspects is also available [here](https://github.com/devitocodes/devito/tree/master/benchmarks/user#a-step-back-configuring-your-machine-for-reliable-benchmarking). 

**\*\*\*\***

## Outline

* [API](#API)
* [Default values](#Default-values)
* [Running example](#Running-example)
* [OpenMP parallelism](#OpenMP-parallelism)
* [The `advanced` mode](#The-advanced-mode)
* [The `advanced-fsg` mode](#The-advanced-fsg-mode)

## API

The optimization level may be changed in various ways:

* globally, through the `DEVITO_OPT` environment variable. For example, to disable all optimizations on all `Operator`'s, one could run with

```
DEVITO_OPT=noop python ...
```

* programmatically, adding the following lines to a program

```
from devito import configuration
configuration['opt'] = 'noop'
```

* locally, as an `Operator` argument

```
Operator(..., opt='noop')
```

Local takes precedence over programmatic, and programmatic takes precedence over global.

The optimization options, instead, may only be changed locally. The syntax to specify an option is

```
Operator(..., opt=('advanced', {<optimization options>})
```

A concrete example (you can ignore the meaning for now) is

```
Operator(..., opt=('advanced', {'blocklevels': 2})
```

That is, options are to be specified _together with_ the optimization level.

## Default values

By default, all `Operator`s are run with the optimization level set to `advanced` and with all options disabled. So this

```
Operator(Eq(...))
```

is equivalent to

```
Operator(Eq(...), opt='advanced')
```

and obviously also to

```
Operator(Eq(...), opt=('advanced', {}))
```

In virtually all scenarios, regardless of application and underlying architecture, this ensures very good performance -- but not necessarily the very best.

## Utilities

The following functions will be used throughout the notebook for printing generated code.

In [1]:
from examples.performance.utils import unidiff_output, print_kernel

## Running example

Throughout the notebook we will generate `Operator`s for the following time-marching `Eq`.

In [2]:
from sympy import sin
from devito import Eq, Grid, Operator, Function, TimeFunction

grid = Grid(shape=(80, 80, 80))

f = Function(name='f', grid=grid)
u = TimeFunction(name='u', grid=grid, space_order=2)

eq = Eq(u.forward, f**2*sin(f)*u.dy.dy)

Despite its simplicity, this `Eq` is all we need to showcase the key components of the Devito optimization engine.

## OpenMP parallelism

There are several ways to enable OpenMP parallelism. The one we use here consists of supplying an **option** to an `Operator`. The next cell illustrates the difference between two `Operator`'s generated with the `noop` optimization level, but with OpenMP enabled on the latter one.

In [3]:
# NOTE: we only need this cell for Continuous Integration, which runs with OpenMP enabled
# This is also an example of how "programmatic takes precedence over global" (see "Default values" paragraphs)
from devito import configuration
configuration['language'] = 'C'

In [4]:
op0 = Operator(eq, opt=('noop'))
op0_omp = Operator(eq, opt=('noop', {'openmp': True}))

# print(op0)
# print(unidiff_output(str(op0), str(op0_omp)))  # Uncomment to print out the diff only
print_kernel(op0_omp)

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(3) schedule(dynamic,1)
    for (int x = x_m; x <= x_M; x += 1)
    {
      for (int y = y_m; y <= y_M; y += 1)
      {
        for (int z = z_m; z <= z_M; z += 1)
        {
          u[t1][x + 2][y + 2][z + 2] = (-(-u[t0][x + 2][y + 2][z + 2]/h_y + u[t0][x + 2][y + 3][z + 2]/h_y)/h_y + (-u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y)/h_y)*sin(f[x + 1][y + 1][z + 1])*pow(f[x + 1][y + 1][z + 1], 2);
        }
      }
    }
  }
  /* End section0 */
  gettimeofday(&end_section0, NULL);
  timers->section0 += (double)(end_section0.tv_sec-start_section0.tv_sec)+(double)(end_section0.tv_usec-start_section0.tv_usec)/1000000;
}


The OpenMP-ized `op0_omp` `Operator` includes:

 - the necessary header file `#include "omp.h"`
 - a `#pragma omp parallel num_threads(nthreads)` directive
 - a `#pragma omp for collapse(...) schedule(dynamic,1)` directive
 
More complex `Operator`'s will have more directives, more types of directives, different iteration scheduling strategies based on heuristics and empirical tuning (e.g., `static` instead of `dynamic`), etc.

We note the OpenMP optimization pass introduces a new symbol, `nthreads`. This allows users to explicitly control the number of threads with which an `Operator` is run.

In [5]:
# NBVAL_IGNORE_OUTPUT
op0_omp.apply(time_M=0)  # Picks up `nthreads` from the standard environment variable OMP_NUM_THREADS
op0_omp.apply(time_M=0, nthreads=2)  # Runs with 2 threads per parallel loop

Operator `Kernel` run in 0.08 s
Operator `Kernel` run in 0.05 s


PerformanceSummary([(PerfKey(name='section0', rank=None),
                     PerfEntry(time=0.04085300000000003, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[]))])

## The `advanced` mode

The default optimization level in Devito is `advanced`. This mode performs several compilation passes to optimize the `Operator` for computation (number of flops), working set size, and data locality.

In the next paragraphs we'll dissect the `advanced` mode to analyze, one by one, some of its key passes.

### Loop blocking

The next cell creates a new `Operator` that adds loop blocking to what we had in `op0_omp`. 

In [6]:
op1_omp = Operator(eq, opt=('blocking', {'openmp': True}))
# print(op1_omp)  # Uncomment to see the *whole* generated code
print_kernel(op1_omp)

void bf0(struct dataobj *restrict f_vec, const float h_y, struct dataobj *restrict u_vec, const int t0, const int t1, const int x0_blk0_size, const int x_M, const int x_m, const int y0_blk0_size, const int y_M, const int y_m, const int z_M, const int z_m, const int nthreads)
{
  float (*restrict f)[f_vec->size[1]][f_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[f_vec->size[1]][f_vec->size[2]]) f_vec->data;
  float (*restrict u)[u_vec->size[1]][u_vec->size[2]][u_vec->size[3]] __attribute__ ((aligned (64))) = (float (*)[u_vec->size[1]][u_vec->size[2]][u_vec->size[3]]) u_vec->data;

  if (x0_blk0_size == 0 || y0_blk0_size == 0)
  {
    return;
  }
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(2) schedule(dynamic,1)
    for (int x0_blk0 = x_m; x0_blk0 <= x_M; x0_blk0 += x0_blk0_size)
    {
      for (int y0_blk0 = y_m; y0_blk0 <= y_M; y0_blk0 += y0_blk0_size)
      {
        for (int x = x0_blk0; x <= x0_blk0 + x0_blk0_size - 1; x += 1)
        {

**\*\* Remark \*\***

`'blocking'` is **not** an optimization level -- it rather identifies a very specific compilation pass. In other words, the `advanced` mode represents a set of passes, and `blocking` is one such pass.

**\*\*\*\***

The `blocking` pass creates additional loops over blocks. In this simple `Operator` there is just one loop nest, so only a pair of additional loops are created. In more complex `Operator`'s, several loop nests may individually be blocked, whereas others may be left unblocked -- this is decided by the Devito compiler according to certain heuristics. The size of a block is represented by the symbols `x0_blk0_size` and `y0_blk0_size`, which are runtime parameters akin to `nthreads`. 

By default, Devito applies 2D blocking and sets the default block shape to 8x8. There are two ways to set a different block shape:

* passing an explicit value. For instance, below we run with a 24x8 block shape

```
op1_omp.apply(..., x0_blk0_size=24)
```

* letting the autotuner pick up a better block shape for us. There are several autotuning modes. A short summary is available [here](https://github.com/devitocodes/devito/wiki/FAQ#devito_autotuning)

```
op1_omp.apply(..., autotune='aggressive')
```

Loop blocking also provides two optimization options:

* `blockinner={False, True}` -- to enable 3D (or any nD, n>2) blocking
* `blocklevels={int}` -- to enable hierarchical blocking, to exploit multiple levels of the cache hierarchy 

In the example below, we construct an `Operator` with six-dimensional loop blocking: the first three loops represent outer blocks, whereas the second three loops represent inner blocks within an outer block.

In [7]:
op1_omp_6D = Operator(eq, opt=('blocking', {'blockinner': True, 'blocklevels': 2, 'openmp': True}))
# print(op1_omp_6D)  # Uncomment to see the *whole* generated code
print_kernel(op1_omp_6D)

void bf0(struct dataobj *restrict f_vec, const float h_y, struct dataobj *restrict u_vec, const int t0, const int t1, const int x0_blk0_size, const int x0_blk1_size, const int x_M, const int x_m, const int y0_blk0_size, const int y0_blk1_size, const int y_M, const int y_m, const int z0_blk0_size, const int z0_blk1_size, const int z_M, const int z_m, const int nthreads)
{
  float (*restrict f)[f_vec->size[1]][f_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[f_vec->size[1]][f_vec->size[2]]) f_vec->data;
  float (*restrict u)[u_vec->size[1]][u_vec->size[2]][u_vec->size[3]] __attribute__ ((aligned (64))) = (float (*)[u_vec->size[1]][u_vec->size[2]][u_vec->size[3]]) u_vec->data;

  if (x0_blk0_size == 0 || y0_blk0_size == 0 || z0_blk0_size == 0)
  {
    return;
  }
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(3) schedule(dynamic,1)
    for (int x0_blk0 = x_m; x0_blk0 <= x_M; x0_blk0 += x0_blk0_size)
    {
      for (int y0_blk0 = y_m; y0_blk0 <= y

### SIMD vectorization

Devito enforces SIMD vectorization through OpenMP pragmas.

In [8]:
op2_omp = Operator(eq, opt=('blocking', 'simd', {'openmp': True}))
# print(op2_omp)  # Uncomment to see the generated code
# print(unidiff_output(str(op1_omp), str(op2_omp)))  # Uncomment to print out the diff only

### Code motion

The `advanced` mode performs a code motion pass. In explicit PDE solvers, this is most commonly used to lift expensive time-invariant sub-expressions out of the inner loops. The pass is however quite general in that it is not restricted to the concept of time-invariance -- any sub-expression invariant with respect to a subset of `Dimension`s is a code motion candidate. In our running example, `sin(f)` gets hoisted out of the inner loops since it is determined to be an expensive invariant sub-expression. In other words, the compiler trades the redundant computation of `sin(f)` for additional storage (the `r1[...]` array).

In [9]:
op3_omp = Operator(eq, opt=('lift', {'openmp': True}))
print_kernel(op3_omp)

float (*r1)[y_size][z_size];
posix_memalign((void**)&r1, 64, sizeof(float[x_size][y_size][z_size]));

struct timeval start_section0, end_section0;
gettimeofday(&start_section0, NULL);
/* Begin section0 */
#pragma omp parallel num_threads(nthreads)
{
  #pragma omp for collapse(3) schedule(static,1)
  for (int x = x_m; x <= x_M; x += 1)
  {
    for (int y = y_m; y <= y_M; y += 1)
    {
      for (int z = z_m; z <= z_M; z += 1)
      {
        r1[x][y][z] = sin(f[x + 1][y + 1][z + 1]);
      }
    }
  }
}
/* End section0 */
gettimeofday(&end_section0, NULL);
timers->section0 += (double)(end_section0.tv_sec-start_section0.tv_sec)+(double)(end_section0.tv_usec-start_section0.tv_usec)/1000000;
for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section1, end_section1;
  gettimeofday(&start_section1, NULL);
  /* Begin section1 */
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp 

### Basic flop-reducing transformations

Among the simpler flop-reducing transformations applied by the `advanced` mode we find:

* "classic" common sub-expressions elimination (CSE),
* factorization,
* optimization of powers

The cell below shows how the computation of `u` changes by incrementally applying these three passes. First of all, we observe how the symbolic spacing `h_y` gets assigned to a temporary, `r0`, as it appears in several sub-expressions. This is the effect of CSE.

In [10]:
op4_omp = Operator(eq, opt=('cse', {'openmp': True}))
print(unidiff_output(str(op0_omp), str(op4_omp)))

--- 
+++ 
@@ -40,7 +40,8 @@
         {
           for (int z = z_m; z <= z_M; z += 1)
           {
-            u[t1][x + 2][y + 2][z + 2] = (-(-u[t0][x + 2][y + 2][z + 2]/h_y + u[t0][x + 2][y + 3][z + 2]/h_y)/h_y + (-u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y)/h_y)*sin(f[x + 1][y + 1][z + 1])*pow(f[x + 1][y + 1][z + 1], 2);
+            float r0 = 1.0/h_y;
+            u[t1][x + 2][y + 2][z + 2] = (-r0*(-r0*u[t0][x + 2][y + 2][z + 2] + r0*u[t0][x + 2][y + 3][z + 2]) + r0*(-r0*u[t0][x + 2][y + 3][z + 2] + r0*u[t0][x + 2][y + 4][z + 2]))*sin(f[x + 1][y + 1][z + 1])*pow(f[x + 1][y + 1][z + 1], 2);
           }
         }
       }



The factorization pass makes sure `r0` is collected to reduce the number of multiplications. In larger, more complex examples, the impact of factorization can be quite significant.

In [11]:
op5_omp = Operator(eq, opt=('cse', 'factorize', {'openmp': True}))
print(unidiff_output(str(op4_omp), str(op5_omp)))

--- 
+++ 
@@ -41,7 +41,7 @@
           for (int z = z_m; z <= z_M; z += 1)
           {
             float r0 = 1.0/h_y;
-            u[t1][x + 2][y + 2][z + 2] = (-r0*(-r0*u[t0][x + 2][y + 2][z + 2] + r0*u[t0][x + 2][y + 3][z + 2]) + r0*(-r0*u[t0][x + 2][y + 3][z + 2] + r0*u[t0][x + 2][y + 4][z + 2]))*sin(f[x + 1][y + 1][z + 1])*pow(f[x + 1][y + 1][z + 1], 2);
+            u[t1][x + 2][y + 2][z + 2] = (r0*(-r0*(-u[t0][x + 2][y + 2][z + 2]) - r0*u[t0][x + 2][y + 3][z + 2]) + r0*(r0*(-u[t0][x + 2][y + 3][z + 2]) + r0*u[t0][x + 2][y + 4][z + 2]))*sin(f[x + 1][y + 1][z + 1])*pow(f[x + 1][y + 1][z + 1], 2);
           }
         }
       }



Finally, `opt-pows` turns costly `pow` calls into multiplications.

In [12]:
op6_omp = Operator(eq, opt=('cse', 'factorize', 'opt-pows', {'openmp': True}))
print(unidiff_output(str(op5_omp), str(op6_omp)))

--- 
+++ 
@@ -41,7 +41,7 @@
           for (int z = z_m; z <= z_M; z += 1)
           {
             float r0 = 1.0/h_y;
-            u[t1][x + 2][y + 2][z + 2] = (r0*(-r0*(-u[t0][x + 2][y + 2][z + 2]) - r0*u[t0][x + 2][y + 3][z + 2]) + r0*(r0*(-u[t0][x + 2][y + 3][z + 2]) + r0*u[t0][x + 2][y + 4][z + 2]))*sin(f[x + 1][y + 1][z + 1])*pow(f[x + 1][y + 1][z + 1], 2);
+            u[t1][x + 2][y + 2][z + 2] = (f[x + 1][y + 1][z + 1]*f[x + 1][y + 1][z + 1])*(r0*(-r0*(-u[t0][x + 2][y + 2][z + 2]) - r0*u[t0][x + 2][y + 3][z + 2]) + r0*(r0*(-u[t0][x + 2][y + 3][z + 2]) + r0*u[t0][x + 2][y + 4][z + 2]))*sin(f[x + 1][y + 1][z + 1]);
           }
         }
       }



### Cross-iteration redundancy elimination (CIRE)

This is perhaps the most advanced among the optimization passes in Devito. CIRE [1] searches for redundancies across consecutive loop iterations. These are often induced by a mixture of nested, high-order, partial derivatives. The underlying idea is very simple. Consider:

```
r0 = a[i-1] + a[i] + a[i+1]
```

at `i=1`, we have

```
r0 = a[0] + a[1] + a[2]
```

at `i=2`, we have

```
r0 = a[1] + a[2] + a[3]
```

So the sub-expression `a[1] + a[2]` is computed twice, by two consecutive iterations.
What makes CIRE complicated is the generalization to arbitrary expressions, the presence of multiple dimensions, the scheduling strategy due to the trade-off between redundant compute and working set, the co-existance with other optimizations (e.g., blocking, vectorization), and much more. All these aspects won't be treated here. What instead we will show is the effect of CIRE in our running example and the optimization options at our disposal to drive the detection and scheduling of the captured redundancies.

In our running example, some cross-iteration redundancies are induced by the nested first-order derivatives along `y`. As we see below, these redundancies are captured and assigned to the two-dimensional temporary `r3`.

Note: the name `cire-sops` means "Apply CIRE to sum-of-product expressions". A sum-of-product is what taking a derivative via finite differences produces.

In [13]:
op7_omp = Operator(eq, opt=('cire-sops', {'openmp': True}))
print_kernel(op7_omp)
# print(unidiff_output(str(op7_omp), str(op0_omp)))  # Uncomment to print out the diff only

float **r5;
posix_memalign((void**)&r5, 64, sizeof(float*)*nthreads);
#pragma omp parallel num_threads(nthreads)
{
  const int tid = omp_get_thread_num();
  posix_memalign((void**)&r5[tid], 64, sizeof(float[y_size + 1][z_size]));
}

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    const int tid = omp_get_thread_num();
    float (*restrict r3)[z_size] __attribute__ ((aligned (64))) = (float (*)[z_size]) r5[tid];
    #pragma omp for collapse(1) schedule(dynamic,1)
    for (int x = x_m; x <= x_M; x += 1)
    {
      for (int y = y_m - 1; y <= y_M; y += 1)
      {
        for (int z = z_m; z <= z_M; z += 1)
        {
          r3[y + 1][z] = -u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y;
        }
      }
      for (int y = y_m; y <=

We note that since there are no redundancies along `x`, the compiler is smart to figure out that `r3` and `u` can safely be computed within the same `x` loop. This is nice -- not only is the reuse distance decreased, but also a grid-sized temporary is avoided.

#### The `min-storage` option

Let's now consider a variant of our running example

In [14]:
eq = Eq(u.forward, f**2*sin(f)*(u.dy.dy + u.dx.dx))
op7_b0_omp = Operator(eq, opt=('cire-sops', {'openmp': True}))
print_kernel(op7_b0_omp)

float (*r6)[y_size + 1][z_size];
posix_memalign((void**)&r6, 64, sizeof(float[x_size + 1][y_size + 1][z_size]));
float (*r5)[y_size + 1][z_size];
posix_memalign((void**)&r5, 64, sizeof(float[x_size + 1][y_size + 1][z_size]));

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(3) schedule(dynamic,1)
    for (int x = x_m - 1; x <= x_M; x += 1)
    {
      for (int y = y_m - 1; y <= y_M; y += 1)
      {
        for (int z = z_m; z <= z_M; z += 1)
        {
          r6[x + 1][y + 1][z] = -u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y;
          r5[x + 1][y + 1][z] = -u[t0][x + 3][y + 2][z + 2]/h_x + u[t0][x + 4][y + 2][z + 2]/h_x;
        }
      }
    }
  }
  #pragma omp parallel num_threads(nthreads)
  {
   

As expected, there are now two temporaries, one stemming from `u.dy.dy` and the other from `u.dx.dx`.  A key difference with respect to `op7_omp` here is that both are grid-size temporaries. This might seem odd at first -- why should the `u.dy.dy` temporary, that is `r6`, now be a three-dimensional temporary when we know already it could be a two-dimensional temporary? This is merely a compiler heuristic: by adding an extra dimension to `r6`, both temporaries can be scheduled within the same loop nest, thus augmenting data reuse and potentially enabling further cross-expression optimizations. We can disable this heuristic through the `min-storage` option.

In [15]:
op7_b1_omp = Operator(eq, opt=('cire-sops', {'openmp': True, 'min-storage': True}))
print_kernel(op7_b1_omp)

float (*r6)[y_size][z_size];
posix_memalign((void**)&r6, 64, sizeof(float[x_size + 1][y_size][z_size]));
float **r8;
posix_memalign((void**)&r8, 64, sizeof(float*)*nthreads);
#pragma omp parallel num_threads(nthreads)
{
  const int tid = omp_get_thread_num();
  posix_memalign((void**)&r8[tid], 64, sizeof(float[y_size + 1][z_size]));
}

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(3) schedule(static,1)
    for (int x = x_m - 1; x <= x_M; x += 1)
    {
      for (int y = y_m; y <= y_M; y += 1)
      {
        for (int z = z_m; z <= z_M; z += 1)
        {
          r6[x + 1][y][z] = -u[t0][x + 3][y + 2][z + 2]/h_x + u[t0][x + 4][y + 2][z + 2]/h_x;
        }
      }
    }
  }
  #pragma omp parallel num_threads(nthreads)
  {

#### The `cire-mincost-sops` option

So far so good -- we've seen that Devito can capture and schedule cross-iteration redundancies. But what if we actually do _not_ want certain redundancies to be captured? There are a few reasons we may like that option, for example we're allocating too much extra memory for the tensor temporaries, and we rather prefer to avoid that. For this, we can tell Devito what the minimum cost of a sub-expression should be in order to be a CIRE candidate. The _cost_ is an integer number based on the operation count and the type of operations:

* A basic arithmetic operation such as `+` and `*` has a cost of 1.
* A `/` has a cost of 25.
* A power with non-integer exponent has a cost of 50.
* A power with non-negative integer exponent `n` has a cost of `n-1` (i.e., the number of `*` it will be converted into).
* A trascendental function (`sin`, `cos`, etc.) has a cost of 50.

The `cire-mincost-sops` option can be used to control the minimum cost of CIRE candidates.

In [16]:
eq = Eq(u.forward, f**2*sin(f)*u.dy.dy)  # Back to original running example
op8_omp = Operator(eq, opt=('cire-sops', {'openmp': True, 'cire-mincost-sops': 107}))
print_kernel(op8_omp)

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(3) schedule(dynamic,1)
    for (int x = x_m; x <= x_M; x += 1)
    {
      for (int y = y_m; y <= y_M; y += 1)
      {
        for (int z = z_m; z <= z_M; z += 1)
        {
          u[t1][x + 2][y + 2][z + 2] = (-(-u[t0][x + 2][y + 2][z + 2]/h_y + u[t0][x + 2][y + 3][z + 2]/h_y)/h_y + (-u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y)/h_y)*sin(f[x + 1][y + 1][z + 1])*pow(f[x + 1][y + 1][z + 1], 2);
        }
      }
    }
  }
  /* End section0 */
  gettimeofday(&end_section0, NULL);
  timers->section0 += (double)(end_section0.tv_sec-start_section0.tv_sec)+(double)(end_section0.tv_usec-start_section0.tv_usec)/1000000;
}


We observe how setting `cire-min-cost=107` makes the tensor temporary produced by `op7_omp` disappear. 106 is indeed the cost of the targeted sub-expression

```
-u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y
```

* One `-` => 1
* One `+` => 1
* Two `/h_y` => 50
* The sub-expression appears `n=2` times with shifted indices (precisely what CIRE looks for), so total cost to be multiplied by `n`.

Note: the cost of integer arithmetic for array indexing is deliberately ignored.

#### The `cire-repeats-sops` option

Higher order derivatives -- typically higher than order two -- may lead to proportionately larger sum-of-products. Consider the following variant of the running example

In [17]:
eq = Eq(u.forward, f**2*sin(f)*u.dy.dy.dy.dy)
op9_omp = Operator(eq, opt=('cire-sops', {'openmp': True}))
print_kernel(op9_omp)

float **r6;
posix_memalign((void**)&r6, 64, sizeof(float*)*nthreads);
#pragma omp parallel num_threads(nthreads)
{
  const int tid = omp_get_thread_num();
  posix_memalign((void**)&r6[tid], 64, sizeof(float[y_size + 1 + 1][z_size]));
}

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    const int tid = omp_get_thread_num();
    float (*restrict r3)[z_size] __attribute__ ((aligned (64))) = (float (*)[z_size]) r6[tid];
    #pragma omp for collapse(1) schedule(dynamic,1)
    for (int x = x_m; x <= x_M; x += 1)
    {
      for (int y = y_m - 1; y <= y_M + 1; y += 1)
      {
        for (int z = z_m; z <= z_M; z += 1)
        {
          r3[y + 1][z] = -(-u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y)/h_y + (-u[t0][x + 2][y + 4][z + 2]/h

At first glance, it's the same story as before, with a two-dimensional tensor temporary extracted and computed in a separate loop nest. By paying closer attention, however, we may note that there are now four occurrences of `r3` in the expression computing `u`, while before, in `op7_omp` for instance, we had only two. Devito seeks for a compromise between compilation speed and run-time performance, so it runs by default searching for redundancies across sum-of-products of `depth=5` or lower. `depth=5` sum-of-products are typically the by-product of second-order derivatives, which is what we find in many PDEs. There are other reasons behind such default value, which for brevity aren't discussed here. To search for deeper sum-of-products (and, therefore, for redundancies induced by higher order derivatives), one shall use the `cire-repeats-sops` option.

In [18]:
op10_omp = Operator(eq, opt=('cire-sops', {'openmp': True, 'cire-repeats-sops': 7}))
print_kernel(op10_omp)

float **r4;
posix_memalign((void**)&r4, 64, sizeof(float*)*nthreads);
#pragma omp parallel num_threads(nthreads)
{
  const int tid = omp_get_thread_num();
  posix_memalign((void**)&r4[tid], 64, sizeof(float[y_size + 1][z_size]));
}

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    const int tid = omp_get_thread_num();
    float (*restrict r2)[z_size] __attribute__ ((aligned (64))) = (float (*)[z_size]) r4[tid];
    #pragma omp for collapse(1) schedule(dynamic,1)
    for (int x = x_m; x <= x_M; x += 1)
    {
      for (int y = y_m - 1; y <= y_M; y += 1)
      {
        for (int z = z_m; z <= z_M; z += 1)
        {
          r2[y + 1][z] = -(-(-u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y)/h_y + (-u[t0][x + 2][y + 4][z + 2]/h_y + u

#### The `cire-mincost-inv` and `cire-repeats-inv` options

Akin to sum-of-products, cross-iteration redundancies may be searched across dimension-invariant sub-expressions, typically time-invariants. So, analogously to what we've seen before:

In [19]:
eq = Eq(u.forward, f**2*sin(f)*u.dy.dy)  # Back to original running example
op11_omp = Operator(eq, opt=('lift', {'openmp': True}))
# print_kernel(op11_omp)  # Uncomment to see the generated code

For convenience, the `lift` pass triggers CIRE for dimension-invariant sub-expressions. As seen before, this leads to the `r1` temporary. By setting a larger value for `cire-mincost-inv`, we avoid a grid-size temporary to be allocated, in exchange for a trascendental function, `sin`, to be computed at each iteration

In [20]:
op12_omp = Operator(eq, opt=('lift', {'openmp': True, 'cire-mincost-inv': 51}))
print_kernel(op12_omp)

for (int time = time_m, t0 = (time)%(2), t1 = (time + 1)%(2); time <= time_M; time += 1, t0 = (time)%(2), t1 = (time + 1)%(2))
{
  struct timeval start_section0, end_section0;
  gettimeofday(&start_section0, NULL);
  /* Begin section0 */
  #pragma omp parallel num_threads(nthreads)
  {
    #pragma omp for collapse(3) schedule(dynamic,1)
    for (int x = x_m; x <= x_M; x += 1)
    {
      for (int y = y_m; y <= y_M; y += 1)
      {
        for (int z = z_m; z <= z_M; z += 1)
        {
          u[t1][x + 2][y + 2][z + 2] = (-(-u[t0][x + 2][y + 2][z + 2]/h_y + u[t0][x + 2][y + 3][z + 2]/h_y)/h_y + (-u[t0][x + 2][y + 3][z + 2]/h_y + u[t0][x + 2][y + 4][z + 2]/h_y)/h_y)*sin(f[x + 1][y + 1][z + 1])*pow(f[x + 1][y + 1][z + 1], 2);
        }
      }
    }
  }
  /* End section0 */
  gettimeofday(&end_section0, NULL);
  timers->section0 += (double)(end_section0.tv_sec-start_section0.tv_sec)+(double)(end_section0.tv_usec-start_section0.tv_usec)/1000000;
}


#### All optimizations together

The `advanced` mode triggers all of the passes we've seen so far... and in fact, many more! Some of them, however, aren't visible in our running example (e.g., all of the MPI-related optimizations). These will be treated in a future notebook.

Obviously, all of the optimization options (e.g., `min-cost`, `cire-mincost-sops`, `blocklevels`) are applicable and composable in any arbitrary way.

In [21]:
op13_omp = Operator(eq, openmp=True)
print(op13_omp)
# op14_omp = Operator(eq, opt=('advanced', {'min-storage': True}))

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
  double section1;
} ;

void bf0(struct dataobj *restrict f_vec, const float h_y, float *restrict r1_vec, struct dataobj *restrict u_vec, const int x_size, const int y0_blk0_size, const int y_size, const int z_size, const int t0, const int t1, const int x0_blk0_size, const int x_M, const int x_m, const int y_M, const int y_m, const int z_M, const int z_m, const int nthreads, float * *restrict r7_vec);

int Kernel(struct dataobj *restrict f_vec, const float h_y, struct dataobj *restrict u_vec, const int x_size, const int y0_blk0_size, const int y_size, const int z_size, const int time_M, const int time_m, struct profiler * timers, const int x0_blk0_size, const int x_

A crucial observation here is that CIRE is applied on top of loop blocking -- the `r5` temporary is computed within the same block as `u`, but with an additional iteration at the block edge along `y` (the first `y` loop starts at `y0_blk0 - 1`, while the second one at `y0_blk0`). Further, the size of `r5` is now a function of the block shape. Altogether, this implements what in the literature is often referred to as "overlapped tiling" (or "overlapped blocking"): data reuse across consecutive loop nests is obtained by cross-loop blocking, which in turn requires a certain degree of redundant computation at the block edge. Clearly, there's a tension between the block shape and the amount of redundant computation. For example, a small block shape guarantees a small(er) working set, and thus better data reuse, but also requires more redundant computation.

## The `advanced-fsg` mode

The alternative `advanced-fsg` optimization level applies the same passes as `advanced`, but in a different order. The key difference is that `advanced-fsg` does not generate overlapped blocking code across CIRE-generated loop nests.

In [22]:
op14_omp = Operator(eq, opt=('advanced-fsg', {'openmp': True}))
print(op14_omp)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
  double section1;
} ;

void bf0(struct dataobj *restrict f_vec, const float h_y, float *restrict r1_vec, struct dataobj *restrict u_vec, const int x_size, const int y_size, const int z_size, const int t0, const int t1, const int x0_blk0_size, const int x_M, const int x_m, const int y_M, const int y_m, const int z_M, const int z_m, const int nthreads, float * *restrict r7_vec);

int Kernel(struct dataobj *restrict f_vec, const float h_y, struct dataobj *restrict u_vec, const int x_size, const int y_size, const int z_size, const int time_M, const int time_m, struct profiler * timers, const int x0_blk0_size, const int x_M, const int x_m, const int y_M, const int y_m, 

The `x` loop here is still shared by the two loop nests, but the `y` one isn't. Analogously, if we consider the alternative `eq` already used in `op7_b0_omp`, we get two completely separate, and therefore individually blocked, loop nests.

In [23]:
eq = Eq(u.forward, f**2*sin(f)*(u.dy.dy + u.dx.dx))
op14_b0 = Operator(eq, opt=('advanced-fsg', {'openmp': True}))
print(op14_b0)

#define _POSIX_C_SOURCE 200809L
#include "stdlib.h"
#include "math.h"
#include "sys/time.h"
#include "xmmintrin.h"
#include "pmmintrin.h"
#include "omp.h"

struct dataobj
{
  void *restrict data;
  int * size;
  int * npsize;
  int * dsize;
  int * hsize;
  int * hofs;
  int * oofs;
} ;

struct profiler
{
  double section0;
  double section1;
} ;

void bf0(const float h_x, const float h_y, float *restrict r7_vec, float *restrict r8_vec, struct dataobj *restrict u_vec, const int t0, const int x0_blk0_size, const int x_M, const int x_m, const int x_size, const int y0_blk0_size, const int y_M, const int y_m, const int y_size, const int z_M, const int z_m, const int z_size, const int nthreads);
void bf1(struct dataobj *restrict f_vec, const float h_x, const float h_y, float *restrict r1_vec, float *restrict r7_vec, float *restrict r8_vec, struct dataobj *restrict u_vec, const int x_size, const int y_size, const int z_size, const int t1, const int x1_blk0_size, const int x_M, const int x_m,

# References

[1] Fabio Luporini, Mathias Louboutin, Michael Lange, Navjot Kukreja, Philipp Witte, Jan Hückelheim, Charles Yount, Paul H. J. Kelly, Felix J. Herrmann, and Gerard J. Gorman. 2020. Architecture and Performance of Devito, a System for Automated Stencil Computation. ACM Trans. Math. Softw. 46, 1, Article 6 (April 2020), 28 pages. DOI:https://doi.org/10.1145/3374916